In [ ]:
# if you need to install - one time run
# %pip install boto3

In [ ]:
# libraies

import datetime
import json
import random
import boto3 # python's AWS library
from botocore.config import Config

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [ ]:
# Exercise

# authenticate between python and AWS

import AWS_credentials

session = boto3.Session(
    aws_access_key_id=AWS_credentials.ACCESS_KEY,
    aws_secret_access_key=AWS_credentials.SECRET_KEY
)

session # check we have authenticated

In [ ]:
# AWS config

my_config = Config(
    region_name = 'eu-west-1',
    signature_version = 'v4',
    retries = {
        'max_attempts': 10,
        'mode': 'standard'
    }
)

In [ ]:
# define Kinesis stream
STREAM_NAME = "StockTradeStream"

#### Stock Price Stream

In [ ]:

def get_data():
    return {
        'EVENT_TIME': datetime.datetime.now().isoformat(),
        'TICKER': random.choice(['AAPL', 'AMZN', 'MSFT', 'INTC', 'TBV']),
        'PRICE': int(random.random() * 100)} # coerced to integer as 
        # know DynamoDB issues with decimals

def generate(stream_name, kinesis_client):
    dynamo_db = boto3.resource('dynamodb',
                               region_name = 'eu-west-1',     
                                aws_access_key_id=AWS_credentials.ACCESS_KEY,
                                aws_secret_access_key=AWS_credentials.SECRET_KEY,
                                config=my_config)
    table = dynamo_db.Table('StockTradesProcessor')
    while True:
        data = get_data() # calls the function above
        print(data) # print the whole streaming dictionary
        kinesis_client.put_record(
            StreamName=stream_name,
            Data=json.dumps(data),
            PartitionKey="partitionkey")
        with table.batch_writer() as batch_writer:
            batch_writer.put_item(Item=data)

In [ ]:
# Exercise - 
# a) call the generate function 
# b) authenticate with kinesis credentials
# c) define the AWS config

In [ ]:
if __name__ == '__main__':
    generate(STREAM_NAME, boto3.client('kinesis',     
                                       aws_access_key_id=AWS_credentials.ACCESS_KEY,
                                       aws_secret_access_key=AWS_credentials.SECRET_KEY,
                                       config=my_config))